In [2]:
import pykep as pk
import numpy as np
import json
import pickle as pkl

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
%matplotlib notebook

In [3]:
import cascade as csc
from copy import deepcopy
from tqdm.notebook import tqdm
import heyoka as hy

# We import the simulation initial conditions and atmospheric density model
The files needed are:
* debris_simulation_ic.pk - created by the notebook 1
* best_fit_density.pk - created by the notebook 1b

In [4]:
with open("data/debris_simulation_ic.pk", "rb") as file:
    r,v,c_radiusl,to_satcat_index,satcat,debris = pkl.load(file)

In [5]:
with open("data/best_fit_density.pk", "rb") as file:
    x_best = pkl.load(file)

In [6]:
def compute_density(h_in_km, best_x):
    """
    returns the heyoka expression for the atmosheric density in kg.m^3. Input is the altitude in km.
    """
    p1 = np.array(best_x[:4])
    p2 = np.array(best_x[4:8])
    p3 = np.array(best_x[8:])
    retval = 0.
    for alpha,beta, gamma in zip(p1,p2, p3):
        retval += alpha*hy.exp(-(h_in_km-gamma)*beta)
    return retval

# We build the dynamical system we want to integrate

In [24]:
# Dynamical variables.
x,y,z,vx,vy,vz = hy.make_vars("x","y","z","vx","vy","vz")

# Constants.
GMe = pk.MU_EARTH
C20 = -4.84165371736e-4
Re = pk.EARTH_RADIUS

# Create Keplerian dynamics.
dyn = csc.dynamics.kepler(mu = GMe)

# Add the J2 terms.
magR2 = hy.sum_sq([x, y, z])
J2term1 = GMe*(Re**2)*np.sqrt(5)*C20/(2*magR2**(1./2))
J2term2 = 3/(magR2**2)
J2term3 = 15*(z**2)/(magR2**3)
fJ2X = J2term1*x*(J2term2 - J2term3)
fJ2Y = J2term1*y*(J2term2 - J2term3)
fJ2Z = J2term1*z*(3*J2term2 - J2term3)
dyn[3] = (dyn[3][0], dyn[3][1] + fJ2X)
dyn[4] = (dyn[4][0], dyn[4][1] + fJ2Y)
dyn[5] = (dyn[5][0], dyn[5][1] + fJ2Z)

In [25]:
rarr = np.array(r)
varr = np.array(v)

In [26]:
sim = csc.sim(rarr[:,0],rarr[:,1],rarr[:,2],varr[:,0],varr[:,1],varr[:,2],radius,0.23 * 806.81,dyn=dyn)

[2022-01-31 15:02:44.925] [cascade] [info] cascade logger initialised


In [29]:
final_t = 86400*365

In [30]:
pbar = tqdm(total=final_t)

while sim.time < final_t:
    orig_time = sim.time
    
    oc = sim.step()
    
    pbar.update(sim.time - orig_time)
   
    if oc == csc.outcome.collision:
        # TODO different code needed for crash
        # on Earth here.
        pi, pj = sim.interrupt_info
        
        print("Collision detected, re-initing particles {} and {}".format(pi, pj))
        
        new_x = deepcopy(sim.x)
        new_y = deepcopy(sim.y)
        new_z = deepcopy(sim.z)
        
        new_vx = deepcopy(sim.vx)
        new_vy = deepcopy(sim.vy)
        new_vz = deepcopy(sim.vz)
        
        new_sizes = deepcopy(sim.sizes)

        for idx in [pi, pj]:
            a = rng.uniform(1.02*Re, 1.3*Re)
            e = rng.uniform(0, 0.02)
            inc = rng.uniform(0, 0.05)
            om = rng.uniform(0, 2*np.pi)
            Om = rng.uniform(0, 2*np.pi)
            nu = rng.uniform(0, 2*np.pi)
            size = rng.uniform(0.01, 0.1)

            r, v = pk.par2ic([a, e, inc, om, Om, nu], GMe)

            new_x[idx] = r[0]
            new_y[idx] = r[1]
            new_z[idx] = r[2]

            new_vx[idx] = v[0]
            new_vy[idx] = v[1]
            new_vz[idx] = v[2]
            
            new_sizes[idx] = size

        sim.set_new_state(new_x, new_y, new_z, new_vx, new_vy, new_vz, new_sizes)
    elif oc == csc.outcome.reentry:
        pi = sim.interrupt_info
        
        print("Particle {} re-entered".format(pi))
        break

pbar.close()
del pbar

  0%|          | 0/31536000 [00:00<?, ?it/s]

Collision detected, re-initing particles 13936 and 15415


NameError: name 'rng' is not defined